In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras.saving import load_model
from keras.models import load_model
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)
from tqdm import tqdm
import pickle

2025-05-04 20:47:44.643634: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-04 20:47:45.005701: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-04 20:47:46.158362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-04 20:47:46.158416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-04 20:47:46.411411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
@tf.keras.utils.register_keras_serializable(package="Custom")
def f2_score(y_true, y_pred):
    # Ensure inputs are tensors
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    # Calculate true positives, false positives, and false negatives
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    # Calculate precision and recall
    epsilon = tf.keras.backend.epsilon()  # Small constant to avoid division by zero
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    # Calculate F2 score
    f2 = (5 * precision * recall) / (4 * precision + recall + epsilon)
    return f2

In [3]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, Flatten, concatenate, MultiHeadAttention, LayerNormalization, Add
from keras.models import Model

In [4]:

@tf.keras.utils.register_keras_serializable(package="Custom")
class DeepSet(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, num_encode, num_decode, **kwargs):
        super(DeepSet, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_encode = num_encode
        self.num_decode = num_decode
        # # Element-wise transformation: phi network
        # self.phi = tf.keras.Sequential([
        #     Dense(self.hidden_dim, activation='relu')
        # ])
        # # Post-aggregation transformation: rho network
        # self.rho = tf.keras.Sequential([
        #     Dense(self.output_dim, activation='relu')
        # ])

        # Element-wise transformation: phi network
        self.phi = tf.keras.Sequential([
            Dense(self.hidden_dim, activation='relu') for _ in range(self.num_encode)
        ])

        # Post-aggregation transformation: rho network
        self.rho = tf.keras.Sequential([
            Dense(self.hidden_dim, activation='relu') for _ in range(self.num_decode - 1)
        ] + [Dense(self.output_dim, activation='relu')])  # Last layer should output correct dimension

    def call(self, x):
        transformed = self.phi(x)  # (batch_size, num_codes, hidden_dim)
        aggregated = tf.reduce_sum(transformed, axis=1)  # (batch_size, hidden_dim)
        output = self.rho(aggregated)  # (batch_size, output_dim)
        return output

    def get_config(self):
        config = super(DeepSet, self).get_config()
        config.update({
            "input_dim": self.input_dim,
            "hidden_dim": self.hidden_dim,
            "output_dim": self.output_dim,
            "num_encode": self.num_encode,
            "num_decode": self.num_decode
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [5]:
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, Flatten, concatenate, MultiHeadAttention, LayerNormalization, Add
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.metrics import AUC, Precision, Recall
import keras.backend as K


@tf.keras.utils.register_keras_serializable(package="Custom")
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [6]:
print(tf.__version__)


2.15.0


In [7]:
new_data_file_path = 'data/NRD_2019_Small.dta'
# Load new data
new_data = pd.read_stata(new_data_file_path)

In [8]:
column_names = new_data.columns.tolist()
print(column_names)

['age', 'died', 'i10_dx1', 'i10_dx2', 'i10_dx3', 'i10_dx4', 'i10_dx5', 'i10_dx6', 'i10_dx7', 'i10_dx8', 'i10_dx9', 'i10_dx10', 'i10_dx11', 'i10_dx12', 'i10_dx13', 'i10_dx14', 'i10_dx15', 'i10_dx16', 'i10_dx17', 'i10_dx18', 'i10_dx19', 'i10_dx20', 'i10_dx21', 'i10_dx22', 'i10_dx23', 'i10_dx24', 'i10_dx25', 'i10_dx26', 'i10_dx27', 'i10_dx28', 'i10_dx29', 'i10_dx30', 'i10_dx31', 'i10_dx32', 'i10_dx33', 'i10_dx34', 'i10_dx35', 'i10_dx36', 'i10_dx37', 'i10_dx38', 'i10_dx39', 'i10_dx40', 'elective', 'female', 'key_nrd', 'i10_birth', 'i10_delivery', 'i10_injury', 'i10_multinjury', 'i10_serviceline', 'pay1', 'pclass_orproc', 'i10_pr16', 'i10_pr17', 'i10_pr18', 'i10_pr19', 'i10_pr20', 'i10_pr21', 'i10_pr22', 'i10_pr23', 'i10_pr24', 'i10_pr25', 'prday16', 'prday17', 'prday18', 'prday19', 'prday20', 'prday21', 'prday22', 'prday23', 'prday24', 'prday25', 'year', 'zipinc_qrtl', 'missing', 'mor30', 'rea30', 'charlindex', 'age_adjust', 'charlindex_age_adjust', 'Index_Readmission', 'Index_Mortality', 

In [10]:

# Define the outcome variable and file path
outcome_var = 'REA30'  # Set outcome variable here, e.g., 'DIED', 'MOR30', 'REA30'

# Load the trained model
model = load_model('Model/readmit_hypertrial_deepset.keras')

# Load the LabelEncoder for ICD codes
with open('./Model/readmit_2016_label_encoder.pkl', 'rb') as file:
    encoder = pickle.load(file)

# Load the MinMaxScaler for 'AGE'
with open('./Model/readmit_2016_age_scaler.pkl', 'rb') as file:
    age_scaler = pickle.load(file)

# Convert all column names to uppercase
new_data.columns = new_data.columns.str.upper()

# Filter out observations where DIED == 1 if outcome_var is REA30
if outcome_var == 'REA30' and 'DIED' in new_data.columns:
    new_data = new_data[new_data['DIED'] != 1]

# Rename columns to match the training data
column_mapping = {
    'AGE': 'AGE',
    'FEMALE': 'FEMALE',
    outcome_var: outcome_var,  # Rename outcome variable
}
icd_column_mapping = {f'I10_DX{i}': f'I10_DX{i}' for i in range(1, 31)}
column_mapping.update(icd_column_mapping)
new_data.rename(columns=column_mapping, inplace=True)

# Define ICD columns
icd_columns = [f'I10_DX{i}' for i in range(1, 36)]

# Encode ICD codes
label_to_int = {label: idx for idx, label in enumerate(encoder.classes_)}
unknown_label_int = 0  # Assign unknown codes to index 0

for col in tqdm(icd_columns, desc="Encoding ICD codes"):
    new_data[col] = new_data[col].astype(str).str.upper()  # Convert to uppercase
    new_data[col] = new_data[col].map(label_to_int).fillna(unknown_label_int).astype(int)

# Normalize 'AGE'
new_data['AGE'] = age_scaler.transform(new_data[['AGE']])

# One-hot encode 'PAY1' and 'ZIPINC_QRTL' only (excluding 'RACE')
new_data = pd.get_dummies(new_data, columns=['PAY1', 'ZIPINC_QRTL'], prefix=['PAY1', 'ZIPINC_QRTL'])

# Ensure that all expected one-hot encoded columns are present
def ensure_columns(data, expected_columns):
    for col in expected_columns:
        if col not in data.columns:
            data[col] = 0
    return data

# Define one-hot encoded columns for 'PAY1' and 'ZIPINC_QRTL' based on training data
pay1_columns = [col for col in new_data.columns if col.startswith('PAY1_')]
zipinc_qrtl_columns = [col for col in new_data.columns if col.startswith('ZIPINC_QRTL_')]

# Ensure all expected columns are present
new_data = ensure_columns(new_data, pay1_columns + zipinc_qrtl_columns)

# Prepare input features
X_new = new_data[['AGE', 'FEMALE'] + pay1_columns + zipinc_qrtl_columns + icd_columns]

# Drop rows with missing values
X_new = X_new.dropna()
if outcome_var in new_data.columns:
    y_new = new_data.loc[X_new.index, outcome_var].dropna()
    X_new = X_new.loc[y_new.index]
else:
    y_new = None

# Convert input features to the required data types
X_new = X_new.astype('float32')

# Make predictions
batch_size = 1024
num_samples = len(X_new)
y_pred_prob = []

# Prepare the inputs in the same structure as during training
for start_idx in tqdm(range(0, num_samples, batch_size), desc="Making Predictions"):
    end_idx = min(start_idx + batch_size, num_samples)
    batch_inputs = [
        X_new[icd_columns].iloc[start_idx:end_idx],
        X_new['AGE'].iloc[start_idx:end_idx].values,
        X_new['FEMALE'].iloc[start_idx:end_idx].values,
    ] + [X_new[col].iloc[start_idx:end_idx].values for col in pay1_columns] \
      + [X_new[col].iloc[start_idx:end_idx].values for col in zipinc_qrtl_columns]

    batch_pred_prob = model.predict(batch_inputs, verbose=0)
    y_pred_prob.extend(batch_pred_prob.flatten())

y_pred_prob = np.array(y_pred_prob)

# Function to calculate the 95% CI for AUC using bootstrapping
def calculate_auc_ci(y_true, y_pred_prob, n_bootstraps=1000, ci=0.95):
    bootstrapped_aucs = []
    np.random.seed(42)

    for i in range(n_bootstraps):
        # Sample with replacement from the data
        indices = np.random.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            # Skip resamples that do not have both classes present
            continue

        score = roc_auc_score(y_true[indices], y_pred_prob[indices])
        bootstrapped_aucs.append(score)

    # Calculate CI
    sorted_aucs = np.sort(bootstrapped_aucs)
    lower_bound = sorted_aucs[int((1 - ci) / 2 * len(sorted_aucs))]
    upper_bound = sorted_aucs[int((1 + ci) / 2 * len(sorted_aucs))]
    return lower_bound, upper_bound

# Evaluate the model if true labels are available
if y_new is not None:
    y_pred = (y_pred_prob > 0.5).astype(int)
    if len(np.unique(y_new)) > 1:
        auc = roc_auc_score(y_new, y_pred_prob)
        accuracy = accuracy_score(y_new, y_pred)
        precision = precision_score(y_new, y_pred)
        recall = recall_score(y_new, y_pred)
        f1 = f1_score(y_new, y_pred)

        # Calculate 95% CI for AUC
        auc_lower, auc_upper = calculate_auc_ci(y_new.values, y_pred_prob)

        print(f"AUC: {auc:.4f}")
        print(f"95% CI for AUC: [{auc_lower:.4f}, {auc_upper:.4f}]")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
    else:
        accuracy = accuracy_score(y_new, y_pred)
        print(f"Accuracy: {accuracy:.4f}")
else:
    print("Predictions:")
    print(y_pred_prob)

# Save predictions to a CSV file
new_data_filtered = new_data.loc[X_new.index].reset_index(drop=True)
new_data_filtered['Predicted_Probability'] = y_pred_prob
if y_new is not None:
    new_data_filtered['Predicted_Label'] = y_pred
new_data_filtered.to_csv('predictions.csv', index=False)


2025-05-04 20:49:07.665065: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/users/xwang259/.conda/envs/tf215_env/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/users/xwang259/.conda/envs/tf215_env/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using 

AUC: 0.7343
95% CI for AUC: [0.7274, 0.7403]
Accuracy: 0.4252
Precision: 0.1659
Recall: 0.9191
F1 Score: 0.2811


In [11]:
f2 = f2_score(y_new, y_pred_prob)
print(f"F2 Score: {f2:.4f}")

F2 Score: 0.4817


In [12]:
optimal_threshold = 0
best_f1 = 0
print(y_pred_prob)
for threshold in np.linspace(0, 1, 100):
    preds = (y_pred_prob > threshold).astype(int)
    f1 = f1_score(y_new, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        optimal_threshold = threshold
print(optimal_threshold)
print(best_f1)

[0.22152314 0.12376571 0.34946087 ... 0.7253336  0.36996785 0.7213385 ]
0.0
0.28109879419369704


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# List of traditional indexes to validate
traditional_indexes = ['INDEX_MORTALITY', 'INDEX_READMISSION', 'CHARLINDEX', 'CHARLINDEX_AGE_ADJUST']
outcome_var = 'REA30'  # Set outcome variable here, e.g., 'DIED', 'MOR30', 'REA30'
new_data_file_path = 'data/NRD_2019_Small.dta'

# Load new data
validation_data = pd.read_stata(new_data_file_path)
# Convert all column names to uppercase
validation_data.columns = validation_data.columns.str.upper()

# Ensure the dataset contains the required variables
missing_columns = [col for col in traditional_indexes + [outcome_var] if col not in validation_data.columns]
if missing_columns:
    raise ValueError(f"The following columns are missing in the validation dataset: {missing_columns}")

# Function to calculate 95% CI for AUC using bootstrapping
def calculate_auc_ci(y_true, y_pred_prob, n_bootstraps=1000, ci=0.95):
    # Convert to NumPy arrays for indexing
    y_true = y_true.to_numpy()
    y_pred_prob = y_pred_prob.to_numpy()

    bootstrapped_aucs = []
    np.random.seed(42)

    for _ in range(n_bootstraps):
        # Generate random sample indices
        indices = np.random.randint(0, len(y_true), len(y_true))

        # Ensure both classes are present in the sample
        if len(np.unique(y_true[indices])) < 2:
            continue

        # Calculate AUC for the bootstrap sample
        score = roc_auc_score(y_true[indices], y_pred_prob[indices])
        bootstrapped_aucs.append(score)

    # Calculate CI bounds
    lower_bound = np.percentile(bootstrapped_aucs, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_aucs, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound

# Initialize a results list
results = []

# Validate each traditional index
for index in traditional_indexes:
    print(f"Validating {index} against {outcome_var}...")

    # Filter data to exclude rows with missing values in the index or outcome variable
    data = validation_data[[index, outcome_var]].dropna()
    y_true = data[outcome_var]
    y_pred_prob = data[index]

    # Convert index scores to binary predictions (if applicable)
    y_pred = (y_pred_prob > 0).astype(int)

    # Ensure both classes are present
    if len(np.unique(y_true)) > 1:
        # Calculate evaluation metrics
        auc = roc_auc_score(y_true, y_pred_prob)
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        f2 = f2_score(y_true, y_pred)

        # Calculate AUC confidence intervals
        auc_lower, auc_upper = calculate_auc_ci(y_true, y_pred_prob)

        # Store results
        results.append({
            'Index': index,
            'AUC': auc,
            'AUC Lower CI': auc_lower,
            'AUC Upper CI': auc_upper,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'F2 Score': float(f2)
        })
    else:
        print(f"Skipped {index}: Only one class present in {outcome_var}")

# Convert results to a DataFrame for easier analysis
if results:
    results_df = pd.DataFrame(results)
    print("\nValidation Metrics for Traditional Scores:")
    print(results_df.to_string(index=False))  # Clean output without indices
else:
    print("No metrics were calculated due to insufficient class diversity in the data.")


Validating INDEX_MORTALITY against REA30...
Validating INDEX_READMISSION against REA30...
Validating CHARLINDEX against REA30...
Validating CHARLINDEX_AGE_ADJUST against REA30...
